In [ ]:
%%bash
pip install pandas-profiling

In [1]:
import os
import pandas as pd
from pandas_profiling import ProfileReport

%matplotlib inline
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

/home/jiewpeng/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:278: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


In [2]:
REGION = 'asia-east1'
BUCKET = '{BUCKET}'
PROJECT = '{PROJECT}'

# Cloud Setup
This section is only required if running on cloud

In [ ]:
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [ ]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

# Data Profiling
Data profiling is done to better understand the data, and to see if there are any invalid data (e.g. out of bounds data, unexpected data types). No data preprocessing should be done here; it should be done in tf.transform so as to have a consistent data pipeline.

In [4]:
df = pd.read_csv('data/raw/spam.csv')[['v1', 'v2']]
df

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [ ]:
ProfileReport(df)

# Split Data
Example uses 80-10-10 split for train, eval and test - change if necessary

In [5]:
RANDOM_SEED = 42
train = df.sample(frac=0.8, random_state=RANDOM_SEED)
eval = df.drop(train.index)
test = eval.sample(frac=0.5, random_state=RANDOM_SEED)
eval = eval.drop(test.index)

In [6]:
def export_datasets(on_cloud=False):
    if on_cloud:
        data_dir = 'gs://{bucket}/spam-classification/data/split'.format(bucket=BUCKET)
    else:
        data_dir = 'data/split'
    
    if not os.path.exists('data'):
        os.mkdir('data')
    if not os.path.exists('data/split'):
        os.mkdir('data/split')
    
    train.to_csv(os.path.join(data_dir, 'train.csv'), index=False)
    eval.to_csv(os.path.join(data_dir, 'eval.csv'), index=False)
    test.to_csv(os.path.join(data_dir, 'test.csv'), index=False)
  
    return
  
export_datasets(on_cloud=False)